In [ ]:
# simple version of the regex matching problem using recursion and 
# no memoization
class Solution:
    def isMatch(self, s: str, p: str) -> bool:
        # goal here is to see if the string s matches the pattern p
        # where p can have '.' as a wildcard for any single character
        # and '*' as a wildcard for matching 0 or more of the
        # preceding character

        # lets do this recursively. Each call will consume 1-2 
        # characters from s and 0 or more characters from p. This way 
        # we only have to worry about a tiny part of the pattern and 
        # string at a time

        def hasWildcard(p: str) -> bool:
            return len(p) >= 2 and p[1] == '*'

        ns = len(s)
        np = len(p)

        # if the string is empty, the pattern must be empty or have wildcards
        if ns == 0:
            if hasWildcard(p):
                return self.isMatch(s, p[2:])
            return np == 0
        
        # if the pattern is empty, the string must also be empty
        if np == 0:
            return ns == 0
        
        if hasWildcard(p):
            # if the next character in the pattern is a wildcard
            # consume characters from the string until we can't 
            # anymore
            
            # match zero times
            return self.isMatch(s, p[2:]) or (s[0] == p[0] or p[0] == '.') and self.isMatch(s[1:], p)
        else:
            # if the next character in the pattern is not a wildcard
            # check for a match and consume the character
            if s[0] == p[0] or p[0] == '.':
                return self.isMatch(s[1:], p[1:])
        
        return False

In [ ]:
# basically the same as above, but with memoization to avoid 
# redundant work
class Solution:
    def isMatch(self, s: str, p: str) -> bool:
        memo = {}  # Cache our results
        
        def hasWildcard(p: str) -> bool:
            return len(p) >= 2 and p[1] == '*'
        
        def match(s: str, p: str) -> bool:
            # If we've seen this combination before, return cached 
            # result
            if (s, p) in memo:
                return memo[(s, p)]
            
            ns = len(s)
            np = len(p)

            # if the string is empty, the pattern must be empty or 
            # have wildcards
            if ns == 0:
                if hasWildcard(p):
                    result = match(s, p[2:])
                else:
                    result = np == 0
                memo[(s, p)] = result
                return result
            
            # if the pattern is empty, the string must also be empty
            if np == 0:
                result = ns == 0
                memo[(s, p)] = result
                return result
            
            if hasWildcard(p):
                # if the next character in the pattern is a wildcard
                # try both possibilities: use it or don't use it
                result = match(s, p[2:]) or ((s[0] == p[0] or p[0] == '.') and match(s[1:], p))
            else:
                # if the next character in the pattern is not a 
                # wildcard must match and consume one character
                if s[0] == p[0] or p[0] == '.':
                    result = match(s[1:], p[1:])
                else:
                    result = False
            
            memo[(s, p)] = result
            return result
        
        return match(s, p)

In [44]:
# ok, so the above is correct and quite efficient, but we can do 
# better! This time we are going to implement the same thing but 
# using recursive dynamic programming with memoization. Basically
# it's going to be the same as above, but using a dict of int tuples
# as the key instead of a string tuple. This will allow us to avoid
# the overhead of creating a string tuple for every recursive call 
# and will make the code a bit faster and more compute/memory 
# efficient

class Solution:
    def isMatch(self, s: str, p: str) -> bool:
        memo = {}
        ns = len(s)
        np = len(p)
    
        # i is the current position in the string s
        # j is the current position in the pattern p
        def dp(i: int, j: int) -> bool:
            if (i, j) in memo:
                return memo[(i, j)]
            
            # if we've reached the end of the pattern, we must 
            # also be at the end of the string
            if j >= np:
                result = i >= ns
                memo[(i,j)] = result
                return result
            
            # check for the existence of a wildcard
            # check if there is a possible match
            hasWildcard = j + 1 < np and p[j + 1] == "*"
            firstMatch = i < ns and (p[j] == "." or s[i] == p[j])

            if hasWildcard:
                # either we don't use the wildcard (skip it) 
                # or use it at least once and try to match more
                result = dp(i, j + 2) or (firstMatch and dp(i+1, j))
            else:
                # match the current character, consume it and move on
                result = firstMatch and dp(i+1, j+1)
            
            memo[(i, j)] = result
            return result
        
        return dp(0, 0)

In [ ]:
# dynamic programming approach. In this case, it is actually better 
# to avoid DP. That's because for non pathological strings, the DP 
# approach is O(ns * np) in both time and space complexity whereas 
# the average case for the recursive approach is ~O(min(ns, np)). The 
# recursive approach is also easier to understand and implement. 
class Solution:
    def isMatch(self, s: str, p: str) -> bool:
        ns = len(s)
        np = len(p)
        
        # dp[i][j] represents if s[i:] matches p[j:]
        dp = [[False] * (np + 1) for _ in range(ns + 1)]
        
        # Empty pattern matches empty string
        dp[ns][np] = True
        
        # Work backwards from the end
        for i in range(ns, -1, -1):
            for j in range(np - 1, -1, -1):
                # check to see if the current characters match
                firstMatch = i < ns and (p[j] == '.' or s[i] == p[j])
                # check to see if the next character is a wildcard
                if j + 1 < np and p[j + 1] == '*':
                    dp[i][j] = dp[i][j + 2] or (firstMatch and dp[i + 1][j])
                else:
                    dp[i][j] = firstMatch and dp[i + 1][j + 1]
                    
        return dp[0][0]

In [47]:
s = "bbc"
p = "b*.*."
result = Solution().isMatch(s, p)
print(f"result: {result}, expected: True")

s = "aa"
p = "a"
result = Solution().isMatch(s, p)
print(f"result: {result}, expected: False")

s = "aa"
p = "a*"
result = Solution().isMatch(s, p)
print(f"result: {result}, expected: True")

s = "abacacccbbbcbcbb"
p = ".*.*.*ab*.*ab.*c*"
result = Solution().isMatch(s, p)
print(f"result: {result}, expected: False")

s = "aaaaaaaaaaaaaaaaaaab"
p = "a*a*a*a*a*a*a*a*a*a*"
result = Solution().isMatch(s, p)
print(f"result: {result}, expected: False")

result: True, expected: True
result: False, expected: False
result: True, expected: True
result: False, expected: False
result: False, expected: False
